In [1]:
from scipy.io import loadmat
import h5py
import numpy as np
import pandas as pd
from PIL import Image

EXP_PATH = "stimulus/nsd_expdesign.mat"
IMAGE_PATH = "stimulus/nsd_stimuli.hdf5"
COCO_MAP = "stimulus/nsd_stim_info_merged.pkl"


def getImages(subj, session, n_images, num_blocks):
    totalImages = n_images * num_blocks
    # Mapping from integer id to NSD id
    mat = loadmat(EXP_PATH)
    subjectim = mat['subjectim'] # 1-indexed
    sessionGroup = (int(session)-1) % 3
    image_indices = subjectim[int(subj)-1][sessionGroup*totalImages : (sessionGroup + 1)*totalImages]
    image_indices = image_indices -1 # img_map is 1-indexed
    sorted_indices = np.argsort(image_indices)
    inverse_indices = np.argsort(sorted_indices)  # To revert back to original order

    # Mapping from NSD id to coco id
    df = pd.read_pickle(COCO_MAP)
    coco_ids = df.iloc[image_indices]['cocoId'].values

    with h5py.File(IMAGE_PATH, 'r') as file:
        dataset = file["imgBrick"]
        sorted_images = dataset[image_indices[sorted_indices], :, :, :]  # Assuming index is within the valid range for dataset # pyright: ignore
        images = sorted_images[inverse_indices]
        pil_images = [Image.fromarray(img) for img in images]

    return pil_images, coco_ids

getImages(1, 1, 208, 16)

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'stimulus/nsd_stimuli.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
df = pd.read_pickle(COCO_MAP)
df

,cocoId,cocoSplit,cropBox,loss,nsdId,flagged,BOLD5000,shared1000,subject1,subject2,...,subject5_rep2,subject6_rep0,subject6_rep1,subject6_rep2,subject7_rep0,subject7_rep1,subject7_rep2,subject8_rep0,subject8_rep1,subject8_rep2
0,532481,val2017,"(0, 0, 0.1671875, 0.1671875)",0.100000,0,False,False,False,0,0,...,0,0,0,0,0,0,0,0,0,0
1,245764,val2017,"(0, 0, 0.125, 0.125)",0.000000,1,False,False,False,0,0,...,0,0,0,0,13985,14176,28603,0,0,0
2,385029,val2017,"(0, 0, 0.125, 0.125)",0.000000,2,False,False,False,0,0,...,0,0,0,0,0,0,0,0,0,0
3,311303,val2017,"(0, 0, 0.16640625, 0.16640625)",0.125000,3,False,False,False,0,0,...,0,0,0,0,0,0,0,0,0,0
4,393226,val2017,"(0, 0, 0.125, 0.125)",0.133333,4,False,False,False,0,0,...,0,13720,22861,23023,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72995,518071,train2017,"(0, 0, 0.125, 0.125)",0.000000,72995,False,False,False,0,0,...,0,0,0,0,0,0,0,6083,11650,26531
72996,255930,train2017,"(0, 0, 0.125, 0.125)",0.125000,72996,False,False,False,0,0,...,0,0,0,0,0,0,0,10402,10434,10625
72997,255934,train2017,"(0, 0, 0.1, 0.1)",0.000000,72997,False,False,False,0,0,...,0,0,0,0,0,0,0,0,0,0
72998,518080,train2017,"(0.125, 0.125, 0, 0)",0.000000,72998,False,False,False,0,0,...,0,0,0,0,5585,11846,14495,0,0,0


In [ ]:
df[df['cocoId'] == 549301]

,cocoId,cocoSplit,cropBox,loss,nsdId,flagged,BOLD5000,shared1000,subject1,subject2,...,subject5_rep2,subject6_rep0,subject6_rep1,subject6_rep2,subject7_rep0,subject7_rep1,subject7_rep2,subject8_rep0,subject8_rep1,subject8_rep2
11159,549301,train2017,"(0, 0.25, 0, 0)",0.0,11159,False,True,True,1,1,...,2420,702,2319,2420,702,2319,2420,702,2319,2420


In [ ]:
subj = 1
session = 1
n_images = 208
num_blocks = 16

totalImages = n_images * num_blocks
# Mapping from integer id to NSD id
mat = loadmat(EXP_PATH)
subjectim = mat['subjectim'] # 1-indexed
sessionGroup = (int(session)-1) % 3
image_indices = subjectim[int(subj)-1][sessionGroup*totalImages : (sessionGroup + 1)*totalImages]
image_indices = image_indices -1 # img_map is 1-indexed
sorted_indices = np.argsort(image_indices)
inverse_indices = np.argsort(sorted_indices)  # To revert back to original order

# Mapping from NSD id to coco id
df = pd.read_pickle(COCO_MAP)
coco_ids = df.iloc[inverse_indices]['cocoId'].values

with h5py.File(IMAGE_PATH, 'r') as file:
    dataset = file["imgBrick"]
    sorted_images = dataset[image_indices[sorted_indices], :, :, :]  # Assuming index is within the valid range for dataset # pyright: ignore
    images = sorted_images[inverse_indices]
    pil_images = [Image.fromarray(img) for img in images]
# return pil_images, coco_ids[inverse_indices]